In [1]:
import pandas as pd
import glob
from tqdm import tqdm
import seaborn as sns

In [2]:
# bases = glob.glob("*.xlsx")
# df = pd.DataFrame()
# for base in tqdm(bases):
#     b = pd.read_excel(base)
#     df = pd.concat([df,b])

In [8]:
# df.to_csv("0_base_agregada.csv", index = False)

In [3]:
df = pd.read_csv("0_base_agregada.csv")

In [4]:
df["FECHA"] = pd.to_datetime(df["FECHA_EMISION"], format='%Y%m%d') #CONVERTIR A FORMATO FECHA
df['WEEK'] = df['FECHA'] - pd.to_timedelta(df['FECHA'].dt.weekday, unit='D') #LUNES DE CADA FECHA (SEMANA)
df['MES'] = df['FECHA'].dt.to_period('M').dt.to_timestamp() #MES DE CADA FECHA (MES)

In [5]:
#RECAUDOS POR SEMANA, PROVINCIA ACTIVIDAD ECONÓMICA
db1 = df.groupby(["WEEK", "PROVINCIA", "ACTIVIDAD_ECONOMICA"])["TOTAL_ITBMS",	"TOTAL_VENTA", "CANTIDAD"].sum().reset_index()
db1["TOTAL_ITBMS"] = db1["TOTAL_ITBMS"] /1000000
db1["TOTAL_VENTA"] = db1["TOTAL_VENTA"] /1000000
db1.head()
db1.to_excel("1_recaudos_semana_provincio_actividadeconomico.xlsx", index = False)

<ipython-input-5-3837b4fb6f56>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  db1 = df.groupby(["WEEK", "PROVINCIA", "ACTIVIDAD_ECONOMICA"])["TOTAL_ITBMS",	"TOTAL_VENTA", "CANTIDAD"].sum().reset_index()
<ipython-input-5-3837b4fb6f56>:6: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  db1.to_excel("1_recaudos_semana_provincio_actividadeconomico.xlsx", index = False)


In [11]:
#NUMERO DE EMPRESAS POR SEMANA
db2 = df.groupby(["MES", "RUC"]).count().reset_index() #NUMERO DE TRANSACCIONES POR SEMANA, NEGOCIO
db2 = db2.groupby(["MES"])["RUC"].count().reset_index() #NUMERO DE NEGOCIOS DISTINTOS POR SEMANA
# db2["CHANGE_RUC"] = db2["RUC"].diff(1)
db2.to_excel("2_empresas_x_mes.xlsx", index = False)

<ipython-input-11-e54dff3c5902>:5: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  db2.to_excel("2_empresas_x_mes.xlsx", index = False)


In [14]:
#ITBM POR SEMANA
db3 = df.groupby(["WEEK"])["TOTAL_ITBMS"].sum().reset_index()
db3.to_excel("3_itbm_x_semana.xlsx", index = False)

<ipython-input-14-4b278455983d>:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.7' currently installed).
  db3.to_excel("3_itbm_x_semana.xlsx", index = False)


In [15]:
#ITBM PER CAPITA
db4 = pd.merge(db2, db3, "inner", "WEEK")
db4["ITMBPP"] = db4["TOTAL_ITBMS"]/db4["RUC"]  #ITMBPP
db4 = db4.iloc[:-2:] #ELIMINAMOS DOS ULTIMAS SEMANAS

#OUTLIERS
p95 = db4["ITMBPP"].quantile(0.95)
media = db4["ITMBPP"].mean()
# db4.loc[db4['ITMBPP'] > p95 , 'ITMBPP'] = media

# db4["ITMBPP_PCT"] = db4["ITMBPP"].pct_change() #CAMBIO PORCENTUAL

# #OUTLIERS
# p95 = db4["ITMBPP_PCT"].quantile(0.95)
# media = db4["ITMBPP_PCT"].mean()
# db4.loc[db4['ITMBPP_PCT'] > p95 , 'ITMBPP_PCT'] = media

# db4.to_excel("4_itbmpp_x_semana.xlsx", index = False)

In [18]:
p95

90898.35464583324

In [16]:
(db4['ITMBPP'] > p95).sum()/len(db4)

0.05314009661835749